In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[2])
import setup
from methods import grid_selection_classifier
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams):
    file = open("results/gridsearch_selection.txt","a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading a dataset
    datasets = [setup.loadCDT, setup.loadCHT]
    ''', setup.load2CDT(path, sep), setup.load2CHT(path, sep),
               setup.load4CR(path, sep), setup.load4CRE_V1(path, sep), setup.load4CRE_V2(path, sep), setup.load5CVT(path, sep),
               setup.loadCSurr(path, sep), setup.load4CE1CF(path, sep), setup.loadUG_2C_2D(path, sep), setup.loadMG_2C_2D(path, sep),
               setup.loadFG_2C_2D(path, sep), setup.loadUG_2C_3D(path, sep), setup.loadUG_2C_5D(path, sep), setup.loadGEARS_2C_2D(path, sep),
               setup.loadKeystroke(path, sep), setup.loadCheckerBoard(path, sep), setup.loadNOAADataset(path, sep), setup.loadElecData(path, sep)'''
    batches=100
    #testing grid search
    for i in range(len(datasets)):
        finalScore = 0
        best_grid={}
        dataValues, dataLabels, description = datasets[i](path, sep)
        sizeOfBatch = int((len(dataValues)-50)/batches)
        print("{}: 100 batches of {} instances".format(description, sizeOfBatch))
        tuned_params = [{"excludingPercentage" : [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5], "K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], "sizeOfBatch":[sizeOfBatch], "batches":[batches]}]
        
        for g in ParameterGrid(tuned_params):
            gs = grid_selection_classifier.proposed_gmm_core_extraction(**g)
            gs.fit(dataValues, dataLabels)
            averageAccuracy = np.mean(gs.predict())
            if finalScore < averageAccuracy:
                finalScore = averageAccuracy
                best_grid = g
        
        print(finalScore)
        print(best_grid)
        
        writeResults(description, finalScore, best_grid)
    
if __name__ == "__main__":
    main()


One Class Diagonal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.8742138365
{'K': 9, 'batches': 100, 'excludingPercentage': 0.65, 'sizeOfBatch': 159}
One Class Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.3773584906
{'K': 3, 'batches': 100, 'excludingPercentage': 0.75, 'sizeOfBatch': 159}
